## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

![alt text](https://antkillerfarm.github.io/images/article/global_average_pooling.png)

* [link text](https://antkillerfarm.github.io/images/article/global_average_pooling.png)

In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

In [0]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [0]:

classifier=Sequential()
input_shape=(32,32,3)

#卷積組合     #32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(filters=3,    
                 kernel_size=(3, 3), 
                 padding='same',
                 input_shape=input_shape))

#卷積組合
classifier.add(Convolution2D(filters=32,    
                 kernel_size=(3, 3), 
                 padding='same'))


# '''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(BatchNormalization(axis=1, 
                             momentum=0.95, 
                             epsilon=0.05))

#flatten
classifier.add(Flatten())

#FC   #output_dim=100,activation=relu
classifier.add(Dense(units=100,
                     activation='relu')) 

#輸出
classifier.add(Dense(units=10,
                     activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam',
                   loss = 'categorical_crossentropy',
                   metrics = ['accuracy'])

classifier.summary()

In [0]:
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

In [0]:
import matplotlib.pyplot as plt

his = classifier.history
plt.figure()

plt.plot(his.history["acc"], "-" ,  label="acc")
plt.plot(his.history["loss"], label="loss")

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [0]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)